# Initialization:

In [9]:
import torch
import dqc
import dqc.xc
import dqc.utils

from DQCAdapter import DQCAdapter

In [5]:
class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a, p):
        super().__init__()
        self.a = a
        self.p = p

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a * rho ** self.p
        
    def get_edensityxc_derivative(self, densinfo, number_of_parameter):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc_derivative(densinfo.u * 2, number_of_parameter) 
                          + self.get_edensityxc_derivative(densinfo.d * 2, number_of_parameter))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            if number_of_parameter == 0: # parameter a
                return rho ** self.p
            elif number_of_parameter == 1: # parameter p
                return self.a * rho ** (self.p - 1)

In [6]:
a = torch.nn.Parameter(torch.tensor(1.0, dtype=torch.double))
p = torch.nn.Parameter(torch.tensor(2.0, dtype=torch.double))
myxc = MyLDAX(a, p)

In [7]:
mol = dqc.Mol(moldesc="N -1 0 0; N 1 0 0", basis="3-21G")
qc = dqc.KS(mol, xc=myxc).run()
ene = qc.energy()
print(ene)

tensor(-54.0932, dtype=torch.float64, grad_fn=<AddBackward0>)


/home/radium/miniconda3/lib/python3.9/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=2.508e-02, |f|=2.288e-01 at iter 32
  warnings.warn(ConvergenceWarning(msg))


In [10]:
adapter = DQCAdapter(qc)

potential info
 tensor([95.5030, 95.5030, 95.5030,  ...,  0.0000,  0.0000,  0.0000],
       dtype=torch.float64, grad_fn=<MulBackward0>)


In [12]:
fockian = adapter.get_fockian()
print(fockian.size())

torch.Size([18, 18])

In [13]:
coefficients = adapter.get_orbital_coefficients()
coefficients.size()

torch.Size([18, 7])

In [14]:
orbital_energies = adapter.get_orbital_energies()
orbital_energies.size()

torch.Size([7])

In [16]:
occupancy = adapter.get_orbital_occupancy()
occupancy.size()

torch.Size([7])

In [30]:
number_of_occupied_orbitals = adapter.get_number_of_occupied_orbitals()

# Calculating adjoint derivatives:

## Calculating $\frac{\partial E[\rho](\vec{\theta})}{\partial \textbf{C}}$

### The first way to calculate derivative from energy with respect to coefficients:
Every type of orbitals can be used

$$ \frac{\partial E[\rho](\vec{\theta})}{\partial C_{bj}} = 2f_b \sum_i C_{bi}F_{ij}$$
so
$$ \frac{\partial E[\rho](\vec{\theta})}{\partial \textbf{C}} = 2\textbf{f}\textbf{C}\textbf{F}$$
$\textbf{f}$ here is matrix: $f_{ab}=\delta_{ab}f_a$

In [25]:
dE_wrt_dC_first_way = 2 * torch.einsum("b,ib,ij->bj", occupancy, coefficients, fockian)
dE_wrt_dC_first_way.size()

torch.Size([7, 18])

### The second way to calculate derivative from energy with respect to coefficients
Only canonical (i.e. eigenfunctions of fockian) orbitals can be used

from the other hand, 
$$ \frac{\partial E[\rho](\vec{\theta})}{\partial C_{bj}} = 2f_b \sum_i C_{bi}F_{ij} = 2f_b \epsilon_b C_{bj}$$
so
$$ \frac{\partial E[\rho](\vec{\theta})}{\partial \textbf{C}} = 2\textbf{f}\epsilon\textbf{C}$$
$\epsilon$ here is matrix: $\epsilon_{ab}=\delta_{ab}\epsilon_a$

In [23]:
dE_wrt_dC_second_way = 2 * torch.einsum("b,b,jb->bj", occupancy, orbital_energies, coefficients)
dE_wrt_dC_second_way.size()

torch.Size([7, 18])

Check that both ways lead to the same results:

In [35]:
print(torch.linalg.matrix_norm(dE_wrt_dC_first_way - dE_wrt_dC_second_way))

tensor(2.7748e-13, dtype=torch.float64, grad_fn=<CopyBackwards>)


## Calculating $\frac{\partial E[\rho](\vec{\theta})}{\partial \vec{\epsilon}}$

$$\frac{\partial E[\rho](\vec{\theta})}{\partial \epsilon_{b}} = 0$$ for all $\epsilon_b$. So:

In [33]:
dE_wrt_depsilon = torch.zeros(number_of_occupied_orbitals)
dE_wrt_depsilon.size()

torch.Size([7])